In [2]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV, KFold
from sklearn.metrics.pairwise import cosine_similarity

In [3]:

plays = pd.read_csv('data/user_artists.dat', sep='\t')
artists = pd.read_csv('data/artists.dat', sep='\t', usecols=['id','name'])

# Merge artist and user pref data
ap = pd.merge(artists, plays, how="inner", left_on="id", right_on="artistID")
ap = ap.rename(columns={"weight": "playCount"})

# Group artist by name
artist_rank = ap.groupby(['name']) \
    .agg({'userID' : 'count', 'playCount' : 'sum'}) \
    .rename(columns={"userID" : 'totalUsers', "playCount" : "totalPlays"}) \
    .sort_values(['totalPlays'], ascending=False)

artist_rank['avgPlays'] = artist_rank['totalPlays'] / artist_rank['totalUsers']
print(artist_rank)

                    totalUsers  totalPlays     avgPlays
name                                                   
Britney Spears             522     2393140  4584.559387
Depeche Mode               282     1301308  4614.567376
Lady Gaga                  611     1291387  2113.563011
Christina Aguilera         407     1058405  2600.503686
Paramore                   399      963449  2414.659148
...                        ...         ...          ...
Morris                       1           1     1.000000
Eddie Kendricks              1           1     1.000000
Excess Pressure              1           1     1.000000
My Mine                      1           1     1.000000
A.M. Architect               1           1     1.000000

[17632 rows x 3 columns]


In [4]:
# Merge into ap matrix
ap = ap.join(artist_rank, on="name", how="inner") \
    .sort_values(['playCount'], ascending=False)
# Preprocessing
pc = ap.playCount
play_count_scaled = (pc - pc.min()) / (pc.max() - pc.min())
ap = ap.assign(playCountScaled=play_count_scaled)

# Build a user-artist rating matrix 
ratings_df = ap.pivot(index='userID', columns='artistID', values='playCountScaled')
ratings = ratings_df.fillna(0).values

# Show sparsity
density = float(len(ratings.nonzero()[0])) / (ratings.shape[0] * ratings.shape[1]) * 100
print("density: %.2f" % density)

density: 0.28


In [5]:
from scipy.sparse import csr_matrix, csc_matrix, coo_matrix

# Build a sparse matrix
X = csr_matrix(ratings)

n_users, n_items = ratings_df.shape
print("rating matrix shape", ratings_df.shape)

user_ids = ratings_df.index.values
artist_names = ap.sort_values("artistID")["name"].unique()

rating matrix shape (1892, 17632)


In [6]:
from lightfm import LightFM
from lightfm.evaluation import auc_score, precision_at_k, recall_at_k
from lightfm.cross_validation import random_train_test_split
from lightfm.data import Dataset

# Build data references + train test
Xcoo = X.tocoo()
data = Dataset()
data.fit(np.arange(n_users), np.arange(n_items))
interactions, weights = data.build_interactions(zip(Xcoo.row, Xcoo.col, Xcoo.data)) 
train, test = random_train_test_split(interactions, random_state=42)

# Ignore that (weight seems to be ignored...)
#train = train_.tocsr()
#test = test_.tocsr()
#train[train==1] = X[train==1]
#test[test==1] = X[test==1]

# To be completed...

/Users/buu/venv/base/lib/python3.8/site-packages/lightfm/_lightfm_fast.py:9: UserWarning: LightFM was compiled without OpenMP support. Only a single thread will be used.
  warnings.warn(


In [7]:
# Train
model = LightFM(learning_rate=0.05, loss='warp', random_state=42)
model.fit(train, epochs=10, num_threads=2)

In [8]:
# Evaluate
train_precision = precision_at_k(model, train, k=10).mean()
test_precision = precision_at_k(model, test, k=10, train_interactions=train).mean()

train_auc = auc_score(model, train).mean()
test_auc = auc_score(model, test, train_interactions=train).mean()

print('Precision: train %.2f, test %.2f.' % (train_precision, test_precision))
print('AUC: train %.2f, test %.2f.' % (train_auc, test_auc))

Precision: train 0.38, test 0.13.
AUC: train 0.96, test 0.86.


In [9]:
# Predict
scores = model.predict(0, np.arange(n_items))
top_items = artist_names[np.argsort(-scores)]
print(top_items)

['Depeche Mode' 'Madonna' 'New Order' ... 'Anata' 'Wayne Marshall'
 'Tokyo Gakuso']


In [10]:
model2 = LightFM(learning_rate=0.08, loss='warp', random_state=42)
model2.fit(train, epochs=10, num_threads=2)

#### Try some parametter and get a board with better

In [11]:
# Evaluate
trp = precision_at_k(model, train, k=10).mean()
tep = precision_at_k(model, test, k=10, train_interactions=train).mean()

tra = auc_score(model, train).mean()
tea = auc_score(model, test, train_interactions=train).mean()

dico = {'Loss':'Warp', 'K':10, 'Learning_rate':0.08, 'Train Precision':trp, 'Test Precision':tep, 'Train Auc':tra,'Test Auc':tea}

tab = pd.DataFrame(dico.items(), columns=['Parameter', 'Values'])

In [12]:
tab

,Parameter,Values
0,Loss,Warp
1,K,10
2,Learning_rate,0.08
3,Train Precision,0.377754
4,Test Precision,0.131981
5,Train Auc,0.964242
6,Test Auc,0.856067


In [13]:
import time

def scoring():

    learning_rate = [0.05, 0.08, 0.10]
    losslist = ['bpr', 'warp', 'logistic', 'warp-kos']
    klist = [5, 7, 9, 11, 13]
    results = []
    
    for x in learning_rate:
        for y in losslist:
            for z in klist:
            
                model = LightFM(learning_rate=x, loss = y)
                t1 = time.process_time()
                model.fit(train, epochs=10, num_threads=2)
                t2 = time.process_time()
                t = t2 - t1
                trainPrecision = precision_at_k(model, train, k=z).mean()
                testPrecision = precision_at_k(model, test, k=z, train_interactions=train).mean()

                trainAUC = auc_score(model, train).mean()
                testAUC = auc_score(model, test, train_interactions=train).mean()

                dicttemp = {}
                dicttemp = {'Time:':t, 'K':z, 'Name':y, 'Learning Rate':x, 'Train Precision':trainPrecision, 'Train AUC':trainAUC, 'Test Precision':testPrecision, "Train AUC":trainAUC, "Test AUC":testAUC}

                results.append(dicttemp)
            
    results = pd.DataFrame(results)

    return results

In [ ]:
scoring()

In [14]:
# Faire la fonction get_recommandation qui prend en entrée un User et renvoie les Artists recommandé 
# (du meilleurs au moins bon au sens du score de recommandation)

In [15]:
def recommend(user):
    scores = model.predict(user, np.arange(n_items))
    top_items = artist_names[np.argsort(-scores)]
    reco = pd.DataFrame(top_items[:10], columns=['Recommandations'])
    return reco

In [16]:
recommend(8)

,Recommandations
0,Radiohead
1,The Beatles
2,David Bowie
3,Pink Floyd
4,Depeche Mode
5,Arctic Monkeys
6,The Smiths
7,Placebo
8,Oasis
9,Interpol


In [17]:
# get_ground_truth qui renvoie les artistes ecoutés par un utilisateur par ordre décroissant du playCountScaled

In [18]:
ap

,id,name,userID,artistID,playCount,totalUsers,totalPlays,avgPlays,playCountScaled
2800,72,Depeche Mode,1642,72,352698,282,1301308,4614.567376,1.000000
35843,792,Thalía,2071,792,324663,26,350035,13462.884615,0.920513
27302,511,U2,1094,511,320725,185,493024,2664.994595,0.909347
8152,203,Blur,1905,203,257978,114,318221,2791.412281,0.731441
26670,498,Paramore,1664,498,227829,399,963449,2414.659148,0.645960
...,...,...,...,...,...,...,...,...,...
38688,913,Destiny's Child,1810,913,1,83,34746,418.626506,0.000000
32955,697,Sia,1290,697,1,56,27597,492.803571,0.000000
71811,4988,Chris Spheeris,510,4988,1,5,3106,621.200000,0.000000
91319,17080,Haylie Duff,1851,17080,1,1,1,1.000000,0.000000


In [19]:
def get_ground_truth(user):
    return ap[(ap.userID == user)][["userID","name","playCountScaled"]].sort_values("playCountScaled", ascending=False)

In [20]:
get_ground_truth(8)

,userID,name,playCountScaled
19523,8,No Angels,0.017834
19528,8,Sandy,0.006791
13176,8,Britney Spears,0.006399
19530,8,Vanessa Petruo,0.004916
14742,8,Beyoncé,0.004037
19531,8,Alexander,0.003986
15139,8,Sugababes,0.002943
19532,8,Queensberry,0.002728
3294,8,Lady Gaga,0.002608
19534,8,Fady Maalouf,0.002319
